In [1]:
import mlflow

mlflow.set_tracking_uri("http://10.125.208.184:5000")

In [2]:
from mlflow.tracking import MlflowClient


In [3]:
client = MlflowClient()


In [5]:
client.transition_model_version_stage(
    name="efficientnetb4",   # 네가 등록한 모델 이름
    version=2,            # 변경할 모델 버전 (v1이면 1)
    stage="Staging"       # "Staging" 또는 "Production"
)


/tmp/ipykernel_529733/143660382.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1745653621522, current_stage='Staging', description='', last_updated_timestamp=1745655083589, name='efficientnetb4', run_id='269385737c1743a7b035ebbe8cf50a75', run_link='', source='mlflow-artifacts:/0/269385737c1743a7b035ebbe8cf50a75/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='2'>

In [12]:
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms

transform_vit = transforms.Compose([
    transforms.Resize((224, 224)),         # ViT 입력 크기 (모델 따라 다를 수도 있음)
    transforms.ToTensor(),                 # 이미지를 Tensor로 변환
    transforms.Normalize(                  # ImageNet Normalize
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, folder_path, transform):
        self.dataset = datasets.ImageFolder(folder_path)
        self.transform = transform
    
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.dataset)

# 데이터셋 정의
train_ds = CustomDataset(TRAIN_PATH, transform=transform_vit)
val_ds   = CustomDataset(VAL_PATH, transform=transform_vit)
test_ds  = CustomDataset(TEST_PATH, transform=transform_vit)

# DataLoader
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False)


In [9]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import random

# 랜덤 시드 고정
torch.manual_seed(42)
random.seed(42)

# 데이터 경로
TRAIN_PATH = "/home/kangkr1002/facial_bone/Training"
VAL_PATH   = "/home/kangkr1002/facial_bone/Validation"
TEST_PATH  = "/home/kangkr1002/facial_bone/Test"

# 공통으로 사용할 transform 정의
transform = transforms.Compose([
    transforms.Resize((380, 380)),   # 380x380 크기 조정
    transforms.ToTensor(),           # Tensor로 변환
])

# 데이터셋 만들기
train_ds = datasets.ImageFolder(root=TRAIN_PATH, transform=transform)
val_ds   = datasets.ImageFolder(root=VAL_PATH, transform=transform)
test_ds  = datasets.ImageFolder(root=TEST_PATH, transform=transform)

# DataLoader로 배치 만들기
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=True)


In [7]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [14]:
import torch
import torch.nn as nn
import mlflow
import mlflow.pytorch
from sklearn.metrics import confusion_matrix, classification_report

# ===== device 세팅 =====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ===== criterion 세팅 =====
criterion = nn.CrossEntropyLoss()

# ===== MLflow 모델 불러오기 =====
mlflow.set_tracking_uri("http://10.125.208.184:5000")

model_name = "VIT_2_Model"
model_uri = f"models:/{model_name}/Staging"

model = mlflow.pytorch.load_model(model_uri)
model = model.to(device)
model.eval()

# ===== Test Set 평가 =====
test_loss = 0.0
test_correct = 0
test_total = 0
test_preds = []
test_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        logits = outputs.logits

        loss = criterion(logits, labels)

        test_loss += loss.item() * images.size(0)
        preds = logits.argmax(dim=1)

        test_total += labels.size(0)
        test_correct += preds.eq(labels).sum().item()

        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# ===== 결과 출력 =====
avg_test_loss = test_loss / test_total
test_accuracy = test_correct / test_total

print(f"Test Loss: {avg_test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

cm = confusion_matrix(test_labels, test_preds)
print("\nConfusion Matrix:")
print(cm)

cls_report = classification_report(test_labels, test_preds, digits=4)
print("\nClassification Report:")
print(cls_report)


Test Loss: 0.3234
Test Accuracy: 0.8750

Confusion Matrix:
[[409  91]
 [ 34 466]]

Classification Report:
              precision    recall  f1-score   support

           0     0.9233    0.8180    0.8674       500
           1     0.8366    0.9320    0.8817       500

    accuracy                         0.8750      1000
   macro avg     0.8799    0.8750    0.8746      1000
weighted avg     0.8799    0.8750    0.8746      1000



In [15]:
from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri("http://10.125.208.184:5000")
client = MlflowClient()

client.transition_model_version_stage(
    name="VIT_2_Model",
    version=1,  # 모델 버전
    stage="Production"
)


/tmp/ipykernel_529733/1670350215.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1745653927217, current_stage='Production', description='', last_updated_timestamp=1745655819475, name='VIT_2_Model', run_id='3754bfc67cc9457b8b665f1cb10d052b', run_link='', source='mlflow-artifacts:/0/3754bfc67cc9457b8b665f1cb10d052b/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='1'>